In [1]:
# Add relevant Jupyter notebook extensions 

In [1]:
# You can double-check your Python path like this...
import sys  
print(sys.path)

['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\python312.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\\Lib', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0', 'c:\\Users\\abdul\\Desktop\\B1\\Scientific Coding\\b1-coding-practical-mt24\\.venv', '', 'c:\\Users\\abdul\\Desktop\\B1\\Scientific Coding\\b1-coding-practical-mt24\\.venv\\Lib\\site-packages', 'c:\\Users\\abdul\\Desktop\\B1\\Scientific Coding\\b1-coding-practical-mt24\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\abdul\\Desktop\\B1\\Scientific Coding\\b1-coding-practical-mt24\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\abdul\\Desktop\\B1\\Scientific Coding\\b1-coding-practical-mt24\\.venv\\Lib\\site-packages\\Pythonwin']


# Simulate closed-loop
After implementing your control functionality, you can simulate the closed-loop with code that looks something like this...

In [31]:
# Import relevant modules
from dynamic import Submarine, ClosedLoop, Mission
from control import PDController

# Instantiate the submarine
sub = Submarine()

# Instantiate the PD Controller with given gains
controller = PDController(kp=0.15, kd=0.6)
# Instantiate your controller (depending on your implementation)
closed_loop = ClosedLoop(sub, controller)
mission = Mission.from_csv(r"C:\Users\abdul\Desktop\B1\Scientific Coding\b1-coding-practical-mt24\data\mission.csv") # You must implement this method in the Mission class

trajectory = closed_loop.simulate_with_random_disturbances(mission)
trajectory.plot_completed_mission(mission)

AttributeError: 'PDController' object has no attribute 'compute_action'